In [2]:
import os
import pandas as pd

# 获取测试数据

In [4]:
data_dir = "data/ruijin_round1_train2_20181022/"

In [5]:
texts    = []
tags     = []

fileidxs = set()
for filename in os.listdir(data_dir):
    fileidxs.add( filename.split('.')[0] )

In [7]:
len(fileidxs)

363

In [8]:
for fileidx in fileidxs:
    with open(data_dir+fileidx+'.txt', 'rb') as f:
        text = f.read().decode('utf-8')
    text_list = [char for char in text]

    tag = pd.read_csv(data_dir+fileidx+'.ann', header=None, sep='\t')
    tag_list = ['O' for _ in range( len(text_list) )]

    for i in range(tag.shape[0]):
        tag_item = tag.iloc[i][1].split(' ')
        cls, start, end = tag_item[0], int( tag_item[1] ), int( tag_item[-1] )

        tag_list[start] = 'B-'+cls
        for j in range(start+1, end):
            tag_list[j] = 'I-'+cls
    assert(len(text_list) == len(tag_list))
    texts.append(text_list)
    tags.append(tag_list)

In [9]:
# write data into file
split_chars = ['。', '！', '？', '，']
train_num   = 0
dev_num     = 0

In [10]:
train_file = 'data/ruijin_train.data'
if os.path.exists(train_file):
    os.remove(train_file)

In [11]:
with open(train_file, 'a') as f:
    for k in range(len(texts) - 1):
        text_ = texts[k]
        tag_  = tags[k]
        for p in range(len(text_)):
            if text_[p] == '\n':
                f.write('LB'+'\t'+tag_[p]+'\n')
            elif text_[p] == ' ':
                f.write('SPACE'+'\t'+tag_[p]+'\n')
            elif text_[p] in split_chars:
                train_num += 1
                f.write(text_[p]+'\t'+tag_[p]+'\n\n')
            else:
                f.write(text_[p]+'\t'+tag_[p]+'\n')

In [14]:
dev_file = 'data/ruijin_dev.data'
if os.path.exists(dev_file):
    os.remove(dev_file)
with open(dev_file, 'a') as f:
    for k in range(len(texts) - 1, len(texts)):
        text_ = texts[k]
        tag_  = tags[k]
        for p in range(len(text_)):
            if text_[p] == '\n':
                f.write('LB'+'\t'+tag_[p]+'\n')
            elif text_[p] == ' ':
                f.write('SPACE'+'\t'+tag_[p]+'\n')
            elif text_[p] in split_chars:
                dev_num += 1
                f.write(text_[p]+'\t'+tag_[p]+'\n\n')
            else:
                f.write(text_[p]+'\t'+tag_[p]+'\n')

In [15]:
print('train_num:{}, dev_num:{}'.format(train_num, dev_num))

train_num:26768, dev_num:136
